### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [ ]:
# poetry add python-dotenv langchain langchain-openai

In [3]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
#load_dotenv(dotenv_path='../.env')
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_L


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="openai/gpt-oss-120b",
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT는 방대한 텍스트 데이터를 이용해 다음에 올 단어를 예측하도록 사전 학습(pre‑training)됩니다. 이때 모델은 입력된 '
 '문맥을 바탕으로 가장 확률이 높은 토큰을 선택하도록 가중치를 조정하며, 수십억 개의 파라미터가 서로 연결되어 언어 패턴을 학습합니다. '
 '이후 인간 피드백을 활용한 강화학습(RLHF) 등으로 미세 조정(fine‑tuning)하여 실제 대화 상황에서 더 안전하고 유용한 답변을 '
 '생성하도록 최적화됩니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="한국어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"한국어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('**ChatGPT 모델의 학습 원리 (3문장)**  \n'
 '1. 대규모 텍스트 데이터를 이용해 Transformer 기반의 신경망을 사전 학습(pre‑training)하여 언어 구조와 의미를 '
 '일반화합니다.  \n'
 '2. 사전 학습된 모델을 실제 대화나 특정 작업에 맞게 인간 피드백을 활용한 강화학습(RLHF)으로 미세 '
 '조정(fine‑tuning)합니다.  \n'
 '3. 학습 과정에서 손실 함수(Loss)를 최소화하도록 가중치를 반복적으로 업데이트해, 입력에 대한 가장 확률이 높은 다음 토큰을 '
 '예측하도록 모델을 최적화합니다.  \n'
 '\n'
 '---\n'
 '\n'
 '### ChatGPT 모델의 주요 장점 요약\n'
 '- **다양한 주제에 대한 높은 이해도**: 방대한 텍스트 코퍼스로 학습돼 일반 상식, 전문 지식, 문화적 맥락 등을 폭넓게 '
 '파악합니다.  \n'
 '- **자연스러운 대화 흐름**: 문맥을 추적하고 이전 발언을 기억해 일관된 대화와 적절한 응답을 생성합니다.  \n'
 '- **빠른 적용성**: 사전 학습 모델을 기반으로 미세 조정만으로도 챗봇, 번역, 요약, 코드 생성 등 다양한 응용 분야에 바로 활용할 '
 '수 있습니다.  \n'
 '\n'
 '---\n'
 '\n'
 '### ChatGPT와 비슷한 AI 모델 (한국어 모델명)\n'
 '- **라마(LLaMA)** – Meta에서 개발한 대규모 언어 모델 시리즈  \n'
 '- **알파

#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [8]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 3},
    {"model_name": "Gemini", "count": 4},
    {"model_name": "claude", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.', 'claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('GPT‑4는 대량의 텍스트 데이터를 사용해 다음 단어를 예측하도록 훈련된 트랜스포머 기반 언어 모델입니다. 학습 과정에서 모델은 입력 '
 '문맥을 인코딩하고, 각 위치에서 가장 확률이 높은 단어를 선택하도록 가중치를 조정합니다. 이를 반복적으로 수행하면서 언어 구조와 의미 '
 '관계를 스스로 학습하게 됩니다.')
<class 'str'> Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Gemini 모델은 대규모 텍스트와 멀티모달 데이터를 사용해 사전 학습(pre‑training) 단계에서 언어와 이미지·음성 등 다양한 '
 '형식의 정보를 동시에 학습합니다.  \n'
 '이후 인간 피드백을 활용한 강화 학습(RLHF) 과정을 거쳐, 모델이 더 정확하고 안전하게 응답하도록 미세 조정합니다.  \n'
 '학습 과정에서는 트랜스포머 기반의 자기‑주의 메커니즘을 이용해 문맥을 장기적으로 파악하고, 여러 모달리티 간의 관계를 통합합니다.  \n'
 '마지막으로 지속적인 업데이트와 평가를 통해 최신 지식과 윤리 기준을 반영하도록 모델을 지속적으로 개선합니다.')
<class 'str'> claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Claude 모델은 대규모 텍스트 데이터를 이용해 **자기 지도 학습(self‑supervised learning)** 방식으로 사전 '
 '학습됩니다.  \n'
 '학습 과정에서는 입력 문장에 마스크를 적용하거나 다음 토큰을 예측하도록 하여, 문맥을 이해하고 자연스러운 언어를 생성하도록 모델을 '
 '최적화합니다.  \n'
 '그 후, 인간 피드백을 활용한 **강화 학습

##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [10]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
## ChatGPT 모델이 어떻게 학습되는지 한눈에 보기  

아래 내용은 **Transformer 기반 대규모 언어 모델**(LLM)인 ChatGPT가 **데이터를 어떻게 받아들이고**, **어떤 목표를 가지고**, **어떤 단계들을 거쳐** 최종적으로 “대화형 AI”가 되는지를 순서대로 정리한 것입니다.  

---

## 1. 기본 구조: Transformer (디코더)  

| 구성 요소 | 역할 |
|----------|------|
| **입력 토큰** | 텍스트 → **토크나이저** → 정수 ID 시퀀스 |
| **임베딩(Embedding)** | 토큰 ID를 고정 차원의 실수 벡터로 변환 |
| **포지셔널 인코딩** | 순서 정보를 벡터에 더해 줌 (sin/cos 혹은 학습 가능한) |
| **Self‑Attention** | 각 토큰이 문맥 전체(앞·뒤)와 상호작용해 의미를 파악 |
| **Feed‑Forward 네트워크** | 비선형 변환으로 표현력을 강화 |
| **Layer Normalization & Residual** | 학습 안정성·깊은 네트워크 전파를 돕는 구조 |
| **출력 헤드** | 마지막 레이어의 벡터 → **Vocabulary** 차원의 로짓 → Softmax → 다음 토큰 확률 분포 |

> **핵심 포인트** : Transformer는 **병렬 처리**

In [11]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
## ChatGPT 모델이 어떻게 학습되는지  
(한국어로 상세히 설명)

---

### 1. 기본 구조: Transformer 아키텍처
- **Transformer**는 2017년 논문 *“Attention Is All You Need”*에서 소개된 모델이며, 현재 대부분의 대형 언어 모델(Large Language Model, LLM)의 핵심이 됩니다.  
- 핵심 요소  
  - **Self‑Attention**: 입력 토큰들 간의 관계를 모두 동시에 고려해 각 토큰이 다른 토큰에 얼마나 “주의(attention)”를 줘야 하는지를 계산합니다.  
  - **멀티‑헤드 어텐션**: 여러 개의 어텐션을 병렬로 수행해 서로 다른 관점을 동시에 학습합니다.  
  - **Feed‑Forward 네트워크**: 각 토큰에 독립적으로 적용되는 두 개の 선형 변환 + 비선형 활성화(보통 GELU)로 구성됩니다.  
  - **Layer Normalization** & **Residual Connection**: 학습 안정성과 깊은 네트워크의 그래디언트 흐름을 돕습니다.

---

### 2. 토큰화 (Tokenization)
- 텍스트는 **토큰**이라는 작은 단위로 변환됩니다.  
- ChatGPT는 **Byte‑Pair Encoding (BPE)** 혹은 **SentencePiece** 같은 서브워드 토크나이저를 사용합니다.  
  - 장점:  
    - 어휘 크기를 수십만 개 수준으로 제한하면서도 드물거나 새로운 단어를 조합해서 표현 가능.  
    - 다국어(특히 한국어)에서도 효율적인 분할이 가능.  

---

### 3. 사전 학습 (Pre‑training) – **자기지도 학습 (Self‑Supervised Learning)**
#### 3.1 목표 함수
- **다음 토큰 예측 (Causal Language Modeling)**:  
  - 입력 시퀀스 `x₁, x₂, …, xₙ`이 주어지면, 모델은 각 위치 `t`에서 `

#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [12]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

## Deep Learning – A Comprehensive Overview

### 1. What is Deep Learning?
Deep learning (DL) is a sub‑field of **machine learning** (ML) that models high‑level abstractions in data by using **artificial neural networks** (ANNs) with many layers—hence the term *deep*. While traditional ML often relies on handcrafted features and relatively shallow models, deep learning lets the system **learn hierarchical representations directly from raw data** (images, text, audio, sensor streams, etc.).

> **In a nutshell:** Deep learning = neural networks with multiple hidden layers that automatically discover useful features from raw inputs, enabling state‑of‑the‑art performance on many perception‑heavy tasks.

---

### 2. Historical Milestones

| Year | Milestone | Why It Matters |
|------|-----------|----------------|
| **1943** | **McCulloch‑Pitts neuron** | First mathematical model of a binary neuron. |
| **1958** | **Perceptron (Rosenblatt)** | Early single‑layer network; showed limits (XOR p

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [13]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
프로그래밍을 사랑합니다.


##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [14]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
**태양계 행성 8개 요약**

| 순서 | 행성 | 평균 거리(태양으로부터) | 지름(≈) | 특징 |
|------|------|----------------------|--------|------|
| 1 | **수성 (Mercury)** | 0.39 AU (약 58 백만 km) | 4,880 km | 가장 작은 행성, 대기 거의 없음, 극심한 온도 차(−173 ℃ ~ +427 ℃) |
| 2 | **금성 (Venus)** | 0.72 AU (≈ 108 백만 km) | 12,104 km | 지구와 비슷한 크기·질량, 두꺼운 이산화탄소 대기와 황산 구름, 온실효과로 표면 온도 ≈ 465 ℃ |
| 3 | **지구 (Earth)** | 1 AU (≈ 150 백만 km) | 12,742 km | 물이 액체 상태로 존재하는 유일한 행성, 풍부한 산소 대기, 하나의 위성(달) |
| 4 | **화성 (Mars)** | 1.52 AU (≈ 228 백만 km) | 6,779 km | 붉은색 표면(산화철), 얇은 CO₂ 대기, 과거 물이 흐른 흔적, 두 개의 작은 위성(포보스·데이모스) |
| 5 | **목성 (Jupiter)** | 5.20 AU (≈ 778 백만 km) | 139,822 km | 가장 큰 행성, 주된 성분은 수소·헬륨, 거대한 폭풍 “대적점”, 79개의 위성(가니메데·이오·유로파·칸소 등) |
| 6 | **토성 (Saturn)** | 9.58 AU (≈ 1.43 억 km) | 116,464 km | 눈에 띄는 고리 시스템(얼음·암석 입자), 가스 행성, 83개의 위성(티탄·레아 등) |
| 7 | **천왕성 (Uranus)** | 19.2 AU (≈ 2.87 억 km) | 50,724 km | 청록색(메탄 가스), 자전축이 거의 98° 기울어져 있어 “옆으로 누운” 행성, 27개의 위성(티타니아·오베론 등) |
| 8 | **해왕성 (Neptune)** | 30.1

##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [17]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(chain)

# 테스트 실행
result = chain.invoke({"input": "양자컴퓨터 정리해 주세요."})
print(result.content)

first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다.'}, {'input': '지구의 대기 구성 요소를 알려주세요.', 'output': '### 지구 대기의 구성\n- **질소 (78%)**: 대기의 대부분을 차지합니다.\n- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.\n- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.\n- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [18]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요? 가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 가을(9‑11월)에는 **여름**에 비해 기온이 서서히 낮아지면서 대기의 대규모 흐름이 바뀌고, 그에 따라 지구과학적인 현상들이 몇 가지 뚜렷하게 나타납니다.  
아래는 **가을에 주로 관측되는 대표적인 지구과학 현상 3가지**와 그 특징을 간단히 정리한 내용입니다.

| # | 현상 | 주요 특징 및 발생 메커니즘 | 가을에 두드러지는 이유 |
|---|------|---------------------------|------------------------|
| 1 | **태풍(열대 저기압) 발생·접근** | • 서쪽 태평양에서 형성된 태풍이 북쪽으로 이동하면서 한반도·동아시아에 영향을 미침.<br>• 바다 표면 온도가 ≈ 26 ℃ 이상이어야 발달 가능하지만, 가을 초입(9‑10월)까지는 아직 충분히 따뜻함.<br>• 태풍은 강풍·폭우· 해일(Storm Surge)을 동반해 강우량·홍수·산사태 위험을 크게 높임. | • 여름 말까지 남아 있는 높은 해수 온도와 남쪽에서 북상하는 **중위도 저기압**(편서풍 급류) 사이의 상호작용이 강해지면서 태풍이 북쪽으로 이동·전환하기 쉬워짐.<br>• 가을 초반은 “태풍 시즌”의 마지막 고점이며, 9월 ~ 10월에 가장 많은 상륙·접근 사례가 보고됨. |
| 2 | **중위도 저기압·전선 활동 강화** | • 가을에는 **극지 고기압**이 남쪽으로 후퇴하고, **극지 저기압**이 북쪽으로 이동하면서 전형적인 중위도 저기압(저기압 전선) 시스템이 활발해짐.<br>• 급격한 온도·습도 차이로 인해 **전선**이 형성되고, 이로 인해 **소나기·뇌우·강수**가 빈번히 발생.<br>• 전선이 지나가면 온도 급변, 바람 전환, 대기압 급락 등 일교차가 크게 늘어남. | • 여름의 **계절풍(남서·남동풍)**이 약해지고, 가을에 들어서면서 **편서풍**이 점차 강화돼 중위도 저기압이 한반도

In [19]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season = get_current_season("south")  # 계절 값 얻기
print(f"현재 계절: {season}")

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.0
)

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season}에 발생하는 자연 현상:\n{response}")

현재 계절: 봄

 봄에 발생하는 자연 현상:
봄에 주로 발생하는 대표적인 지구과학 현상 3가지는 다음과 같습니다.

1.  **춘분(춘분점)**: 춘분은 지구의 춘분점으로 북반구에서 봄이 시작되는 날입니다. 태양이 남춘분점(적도 남쪽)에서 북춘분점(적도 북쪽)으로 이동할 때 발생하며, 낮과 밤의 길이가 거의 같습니다.
2.  **빙하 융해**: 빙하 융해는 빙하의 온도가 상승하면서 빙하의 표면이나 내부에서 눈이나 얼음이 녹는 현상입니다. 이로 인해 바다 수위가 상승하고, 생태계의 변화가 일어날 수 있습니다.
3.  **황사 현상**: 황사 현상은 중국이나 몽골의 사막 지역에서 강한 바람이 불어오는 때 발생합니다. 이로 인해 미세먼지가 많이 발생하여 건강에 해를 끼칠 수 있습니다. 이 현상은 특히 봄철에 더욱 심해지며, 아시아 지역 전반에 영향을 미칩니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [20]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1377.98원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: ## 2024년 4월 5일 환율 분석: 1달러 = 1377.98원

### 1. 현재 환율 상태

2024년 4월 5일, 1달러는 1377.98원으로 거래되고 있습니다. 이는 한국 원화의 가치가 미국 달러화 대비 약세임을 나타냅니다. 

### 2. 최근 환율 변동 추세

최근의 글로벌 경제 상황과 통화 정책은 환율에 큰 영향을 미치고 있습니다. 특히, 미국의 금리 정책과 한국의 경제 성장률, 무역 수지 등이 환율 변동에 중요한 요소입니다.

### 3. 원화 약세 원인

1. **글로벌 경제 불확실성**: 글로벌 경제의 불확실성이 증가할 때, 투자자들은 안전 자산으로 여겨지는 달러화로 자금을 이동시키는 경향이 있습니다. 이는 달러화의 수요를 증가시키고, 원화의 가치를 하락시키는 요인입니다.

2. **미국 연방준비제도(Fed)의 금리 정책**: 미국 Fed의 금리 인상이나 인하 여부는 달러화 가치에 큰 영향을 미칩니다. 최근 미국 경제 지표와 Fed의 정책 발표에 따라 달러화의 가치가 변동하고 있습니다.

3. **한국의 경제 상황**: 한국의 수출입 실적, 무역 수지, 경제 성장률 등도 원화 가치에 영향을 주는 중요한 요소입니다. 특히, 반도체와 같은 주요 수출 품목의 수요 변동은 한국의 경제 상황과 원화 가치에 큰 영향을 미칠 수 있습니다.

### 4. 향후 환율 전망

향후 환율은 다양한 경제적 요인에 의해 변동할 것입니다. 

1. **경제 지표 발표**: 미국과 한국의 경제 지표 발표는 향후 환율 변동에 중요한 영향을 미칠 것입니다. 특히, 고용률, 물가 상승률, 제조업 활동 등 주요 경제 지표에 주목할 필요가 있습니다.

2. **중앙은행의 통화 정책**: 미국 Fed와 한국은행의 통화 정책 결정도 환율에 큰 영향을 미칠 것입니다. 금리 인상이나 인하, 양적 긴축 또는 완화 정책 등은 달러화와 원화의 상대적 가치에 영향